In [1]:
import torch
from PIL import Image
import numpy as np
from diffusers import StableDiffusionPipeline
from research.ip_adapter_module import IPAdapterModule
from safetensors.torch import load_file
# 1. Настройки
device = "cuda" if torch.cuda.is_available() else "cpu"
ckpt_path = "C:/comfy/ComfyUI_windows_portable/ComfyUI/models/checkpoints/dreamshaper_8.safetensors"   # или путь к SDXL
adapter_ckpt = "C:/comfy/ComfyUI_windows_portable/ComfyUI/models/ipadapter/ip-adapter-plus_sd15.safetensors"
clip_repo = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"

# 2. Инициализация пайплайна и адаптера
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
unet = pipe.unet
state_dict = load_file(ckpt_path)  # возвращает словарь {param_name: tensor}

# Фильтруем и загружаем только веса UNet
unet_keys = [k for k in state_dict.keys() if k.startswith("model.diffusion_model.")]
mapped_state_dict = {
    k.replace("model.diffusion_model.", ""): v for k, v in state_dict.items() if k.startswith("model.diffusion_model.")
}
missing, unexpected = unet.load_state_dict(mapped_state_dict, strict=False)
print(f"Missing: {missing}, Unexpected: {unexpected}")




Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Missing: ['conv_in.weight', 'conv_in.bias', 'time_embedding.linear_1.weight', 'time_embedding.linear_1.bias', 'time_embedding.linear_2.weight', 'time_embedding.linear_2.bias', 'down_blocks.0.attentions.0.norm.weight', 'down_blocks.0.attentions.0.norm.bias', 'down_blocks.0.attentions.0.proj_in.weight', 'down_blocks.0.attentions.0.proj_in.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.norm1.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.norm1.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.norm2.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.norm2.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.at

In [3]:
adapter = IPAdapterModule(
    adapter_ckpt=adapter_ckpt,
    image_encoder_repo=clip_repo,
    device=device,
    num_tokens=16,
    is_plus=True,
    cross_attention_dim=768,
    layers_to_patch=[3]  # патчим только слой с индексом 3
)

# 3. Патчинг UNet
adapter.patch_unet(pipe.unet)

# 4. Готовим входные данные
prompts = ["A futuristic cityscape", "A serene mountain lake"]
# загрузим два изображения для image-conditioning
img1 = Image.open("left.jpg").convert("RGB")
img2 = Image.open("right.jpg").convert("RGB")

# создаём маски — левую и правую половину
w, h = img1.size
mask_left  = torch.from_numpy(np.tile([[1]* (w//2) + [0]* (w - w//2)], (h,1))).float()
mask_right = 1 - mask_left

# повторяем в списках
image_prompts = [img1, img2]
masks = [mask_left, mask_right]

# 5. Запуск генерации
images = adapter.generate_batch(
    pipe=pipe,
    prompts=prompts,
    image_prompts=image_prompts,
    masks=masks,
    sigma_range=(1.0, 0.5),   # применять адаптер с шага 0→0.5
    guidance_scale=7.5,
    num_inference_steps=30,
    seed=42
)

# 6. Сохраняем результат
for i, im in enumerate(images):
    im.save(f"output_{i}.png")

RuntimeError: Error(s) in loading state_dict for CLIPVisionModelWithProjection:
	size mismatch for visual_projection.weight: copying a param with shape torch.Size([1024, 1280]) from checkpoint, the shape in current model is torch.Size([768, 1280]).
	You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.